In [1]:
import os


In [2]:
%pwd

'c:\\Users\\kushagra\\Kidney-Disease_Classification-MLflow-DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\kushagra\\Kidney-Disease_Classification-MLflow-DVC'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:

import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [9]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [10]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2026-02-21 16:22:52,145: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-02-21 16:22:52,194: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-21 16:22:52,203: INFO: common: created directory at: artifacts]
[2026-02-21 16:22:52,209: INFO: common: created directory at: artifacts\training]


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'artifacts\\data_ingestion\\kidney-ct-scan-image'

In [11]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2026-02-21 16:28:40,902: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-02-21 16:28:40,924: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-21 16:28:40,932: INFO: common: created directory at: artifacts]
[2026-02-21 16:28:40,939: INFO: common: created directory at: artifacts\training]


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'artifacts\\data_ingestion\\kidney-ct-scan-image'

In [12]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2026-02-21 16:30:09,999: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-02-21 16:30:10,010: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-21 16:30:10,013: INFO: common: created directory at: artifacts]
[2026-02-21 16:30:10,018: INFO: common: created directory at: artifacts\training]
Found 2487 images belonging to 4 classes.
Found 9959 images belonging to 4 classes.


InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "d:\anoconda3\envs\kidney\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "d:\anoconda3\envs\kidney\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "d:\anoconda3\envs\kidney\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "d:\anoconda3\envs\kidney\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "d:\anoconda3\envs\kidney\lib\site-packages\ipykernel\kernelapp.py", line 758, in start
      self.io_loop.start()
    File "d:\anoconda3\envs\kidney\lib\site-packages\tornado\platform\asyncio.py", line 211, in start
      self.asyncio_loop.run_forever()
    File "d:\anoconda3\envs\kidney\lib\asyncio\base_events.py", line 603, in 

In [13]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2026-02-21 16:36:27,821: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-02-21 16:36:27,847: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-21 16:36:27,857: INFO: common: created directory at: artifacts]
[2026-02-21 16:36:27,861: INFO: common: created directory at: artifacts\training]
Found 2487 images belonging to 4 classes.
Found 9959 images belonging to 4 classes.


InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "d:\anoconda3\envs\kidney\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "d:\anoconda3\envs\kidney\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "d:\anoconda3\envs\kidney\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "d:\anoconda3\envs\kidney\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "d:\anoconda3\envs\kidney\lib\site-packages\ipykernel\kernelapp.py", line 758, in start
      self.io_loop.start()
    File "d:\anoconda3\envs\kidney\lib\site-packages\tornado\platform\asyncio.py", line 211, in start
      self.asyncio_loop.run_forever()
    File "d:\anoconda3\envs\kidney\lib\asyncio\base_events.py", line 603, in 